In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import tarfile

In [2]:
f = pd.read_csv('prudhoe_15.csv')

/scratch/richardso21/conda/envs/fgsegnet/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
f.head()

,File,RelativePath,Folder,Date,Time,ImageQuality,DeleteFlag,Caribou,Bear,Fox,...,Comment,Snow_Cover,Camera_Failure,Not_Counted,First_Calf,Waterfowl,Upland_Game_Bird,Songbird,Sea_Shore_Bird,Unnamed: 24
0,RCNX0001.JPG,S15_5.5-6.9,15,05-May-2019,19:51:54,Ok,False,0,0,0,...,NaN,NaN,NaN,False,False,False,False,False,False,NaN
1,RCNX0002.JPG,S15_5.5-6.9,15,05-May-2019,19:52:04,Ok,False,0,0,0,...,NaN,NaN,NaN,False,False,False,False,False,False,NaN
2,RCNX0003.JPG,S15_5.5-6.9,15,05-May-2019,19:52:14,Ok,False,0,0,0,...,NaN,NaN,NaN,False,False,False,False,False,False,NaN
3,RCNX0004.JPG,S15_5.5-6.9,15,05-May-2019,19:55:00,Ok,False,0,0,0,...,Season_Start,99.0,NaN,False,False,False,False,False,False,NaN
4,RCNX0005.JPG,S15_5.5-6.9,15,05-May-2019,20:00:00,Ok,False,0,0,0,...,NaN,NaN,NaN,False,False,False,False,False,False,NaN


In [9]:
f.RelativePath.unique()

array(['S12_5.5_6.8', 'S12_6.9-7.6', 'S12_7.6-8.9', 'S12_8.9-9.4'],
      dtype=object)

In [11]:
# relative_path = {}
# folders = ["2019/100RECNX", "2019/101RECNX", "2019_Summer/100RECNX", "2019_Summer/101RECNX"]
# for u, v in zip(f.RelativePath.unique(), folders):
#     relative_path[u] = v

In [5]:
relative_path = {
    "S15_5.5-6.9": "2019/100RECNX",
    "S15_6.9-7.6": "2019/101RECNX",
    "S15_7.6-8.10": "2019_Summer/100RECNX",
    "S15_8.10-9.4": "2019_Summer/101RECNX"
}

In [6]:
def get_abs_path(pth, file):
    return os.path.join('/tank/data/nna/cameraTrap/prudhoe/15', relative_path[pth], file)

In [13]:
f = pd.read_csv('csv_raw/prudhoe_12.csv')

/scratch/richardso21/conda/envs/fgsegnet/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (12,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
f.Bird.unique()

array([nan, 'Bird of Prey', 'Upland Game Bird', 'Songbrid', 'Seabird',
       'Shorebird'], dtype=object)

In [17]:
f.iloc[:,12]

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
35811    NaN
35812    NaN
35813    NaN
35814    NaN
35815    NaN
Name: Flag_Image, Length: 35816, dtype: object

In [2]:
categories = ["Caribou", "Bear", "Fox", "Wolf", "Muskox",
              "Misc_Land_Animal", "Human", "Other", "Waterfowl",
              "Upland_Game_Bird", "Songbird", "Sea_Shore_Bird"]

In [5]:
fs = []

In [8]:
for i, row in tqdm(f.iterrows()):
    for c in categories:
        try:
            if row[c] > 0:
                fs.append([row["File"],
                           relative_path[row["RelativePath"]],
                           get_abs_path(row["RelativePath"], row["File"]),
                           c])
        except KeyError:
            pass

4141it [00:01, 3746.02it/s]


NameError: name 'relative_path' is not defined

In [28]:
fs[0]

['RCNX0032.JPG',
 '2019/100RECNX',
 '/tank/data/nna/cameraTrap/prudhoe/15/2019/100RECNX/RCNX0032.JPG',
 'Human']

In [29]:
fs_df = pd.DataFrame(fs, columns=["fileName", "RelFilePath", "AbsFilePath", "Type"])

In [30]:
fs_df.head()

,fileName,RelFilePath,AbsFilePath,Type
0,RCNX0032.JPG,2019/100RECNX,/tank/data/nna/cameraTrap/prudhoe/15/2019/100R...,Human
1,RCNX0533.JPG,2019/100RECNX,/tank/data/nna/cameraTrap/prudhoe/15/2019/100R...,Caribou
2,RCNX0534.JPG,2019/100RECNX,/tank/data/nna/cameraTrap/prudhoe/15/2019/100R...,Caribou
3,RCNX0535.JPG,2019/100RECNX,/tank/data/nna/cameraTrap/prudhoe/15/2019/100R...,Caribou
4,RCNX0536.JPG,2019/100RECNX,/tank/data/nna/cameraTrap/prudhoe/15/2019/100R...,Caribou


In [31]:
fs_df.to_csv('prudhoe_15_parsed.csv')

In [37]:
tf = tarfile.open('prudhoe_15_positive.tar.gz', mode='x:gz')

In [38]:
e = 0
for i, row in fs_df.iterrows():
    try:
        tf.add(row["AbsFilePath"], arcname=os.path.join(row["RelFilePath"],row["fileName"]))
#         break
    except:
        e += 1
print(f'{e} error(s) encountered. [Probably missing files]')

120 error(s) encountered. [Probably missing files]


In [39]:
tf.close()